In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
data1 = pd.read_csv('result/rsrp-l2100-alljabo-40x40-pop.csv/0.part')
data2 = pd.read_csv('result/rsrp-l2100-alljabo-40x40.csv/0.part')

existing_sites = pd.read_excel('sites/Existing Sites-L2100.xlsx')
new_sites = pd.read_csv('sites/1657 Sites.csv')

kecamatan = gpd.read_file('Polygon XL - EID Area/Kecamatan_Jabotabek.TAB')

In [3]:
data1.rename(columns={'rsrp-l2100':'rsrp-l2100-qty'}, inplace=True)

In [4]:
existing_sites = existing_sites[['Tower ID', 'Frequency System', 'Longitude', 'Latitude']]
new_sites = new_sites[['Planning ID','Lat', 'Long']]

In [5]:
data2.rename(columns={'geometry_polygon': 'geometry_polygon1'}, inplace=True)
data1 = pd.concat([data1, data2], axis=1, join='inner')

In [6]:
data1

,geometry_polygon,rsrp-l2100-qty,geometry_polygon1,rsrp-l2100
0,"POLYGON ((105.176039 -6.203633, 105.176398 -6....",75.0,"POLYGON ((105.146209 -6.293123, 105.146568 -6....",-103.5
1,"POLYGON ((105.280265 -6.454494, 105.280624 -6....",1.0,"POLYGON ((105.274514 -6.558001, 105.274874 -6....",-120.0
2,"POLYGON ((105.31297 -6.599692, 105.31333 -6.59...",1.0,"POLYGON ((105.274874 -6.558001, 105.275233 -6....",-121.0
3,"POLYGON ((105.31333 -6.599332, 105.313689 -6.5...",1.0,"POLYGON ((105.27739 -6.556564, 105.277749 -6.5...",-119.0
4,"POLYGON ((105.322315 -6.589269, 105.322674 -6....",2.0,"POLYGON ((105.280265 -6.568783, 105.280624 -6....",-115.0
...,...,...,...,...
2215686,"POLYGON ((107.189757 -6.280544, 107.190116 -6....",267.0,"POLYGON ((107.636132 -6.087547, 107.636491 -6....",-125.0
2215687,"POLYGON ((107.189757 -6.280904, 107.190116 -6....",240.0,"POLYGON ((107.636132 -6.123487, 107.636491 -6....",-114.0
2215688,"POLYGON ((107.189757 -6.281263, 107.190116 -6....",6.0,"POLYGON ((107.636491 -6.123487, 107.636851 -6....",-116.0
2215689,"POLYGON ((107.189757 -6.281623, 107.190116 -6....",5.0,"POLYGON ((107.641523 -6.14541, 107.641882 -6.1...",-115.0


In [7]:
# data1['compare'] = data1['geometry_polygon'] == data1['geometry_polygon1']
data1.reset_index(drop=True, inplace=True)
data1.drop(columns=['geometry_polygon1'], inplace=True)
data1 = data1[data1['rsrp-l2100-qty'] > 100] 
data1 = data1[data1['rsrp-l2100'] < -105]

In [8]:
data1   = gpd.GeoDataFrame(data1, geometry=gpd.GeoSeries.from_wkt(data1['geometry_polygon']), crs=4326)
existing_sites = gpd.GeoDataFrame(existing_sites, geometry=gpd.points_from_xy(existing_sites['Longitude'], existing_sites['Latitude']), crs=4326)
new_sites = gpd.GeoDataFrame(new_sites, geometry=gpd.points_from_xy(new_sites['Long'], new_sites['Lat']), crs=4326)

In [9]:
data1 = data1.sjoin_nearest(existing_sites, how='left', max_distance = 0.0008985 * 5, distance_col='distance_existing')
data1.drop(columns=['index_right'], inplace=True)
data1 = data1.sjoin_nearest(new_sites, how='left', max_distance = 0.0008985 * 5, distance_col='distance_new')

/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/geopandas/array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/home/nivag/2023-Linux/.python-3.10/lib/python3.10/site-packages/geopandas/array.py:364: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [10]:
data1.reset_index(inplace=True)

In [11]:
data1.drop(columns=['index'], inplace=True)

In [12]:
data1.rename(columns={'rsrp-l2100':'rsrp-qty-sample', 'rsrp':'rsrp-level-avg'}, inplace=True)

In [13]:
data1.drop(columns=['index_right'], inplace=True)

In [14]:
data1 = gpd.sjoin(left_df= data1, right_df=kecamatan, how='left', predicate='within')

In [15]:
data1.to_csv('result/result-bad-grid-l2100.csv', index=False)